In [1]:
import pandas as pd
from fuzzywuzzy import process, fuzz

/Users/kjaymiller/.pyenv/versions/3.9.1/envs/diversity/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_json('updated_diversity_orgs.json').fillna('')

ValueError: Expected object or value

In [ ]:
df.info()

In [ ]:
for col in df[['name','city']]:
    df[col] = df[col].str.strip()
    print('Number of unique values in ' + str(col) +': ' + str(df[col].nunique()))

In [ ]:
# This shows where duplicates may exist
def replacements(df=df):
    """generate a list of unique values group them based on similarity"""
    unique_city = df['city'].unique().tolist()
    score_sort = [(x,) + i
                 for x in unique_city 
                 for i in process.extract(x, unique_city, scorer=fuzz.token_sort_ratio)]
    #Create a dataframe from the tuples
    similarity_sort = pd.DataFrame(score_sort, columns=['city_sort','match_sort','score_sort'])
    return similarity_sort[similarity_sort['score_sort'].between(80, 99)]

In [ ]:
groups = replacements().groupby(by=['match_sort'])
groups.count()[groups.count()['city_sort'] >= 1]

In [ ]:
# Get the List and Check Against a City
city = 'New York'
unique_list = replacements()
unique_list[unique_list['match_sort'].str.contains(city)]

In [ ]:
def unify(df, match_series, city_sort_value):
    """
    given a dataframe(df) and match_series,
    set all values of the dataframe to the city_sort_value
    """
    matcher = match_series[match_series['match_sort'] == city_sort_value]['city_sort']
    df.loc[df['city'].isin(matcher), ['city']] = city_sort_value

In [ ]:
cities = [
    'Washington, DC',
    'Atlanta, GA',
    'Austin, TX',
    'Boston, MA',
    'Los Angeles, CA'
    'Houston, TX',
    'Cleveland, OH',
    'San Diego, CA',
    'Dallas, TX',
    'Portland, OR',
    'Raleigh-Durham, NC',
    'San Francisco, CA',
    'Seattle, WA',
    'Rio De Janeiro, Brazil',
    'London, UK',
    'Twin Cities, USA',
    'São Paulo, Brazil',
    'Tel Aviv, Israel',
    'New York City, NY'
    ]

In [ ]:
# Choose a value to match against and change all found values to that assigned value
for city in cities:
    unify(df, unique_list, city)

In [ ]:
# To Make a Brute Force change
df.loc[df['city'].str.contains('London'), ['city']] = "London, United Kingdom"

In [ ]:
# To check a value in the DF
df[df['city'].str.contains("London")]

In [ ]:
# Save your Work
df.to_json('updated_diversity_orgs.json', orient="records")